In [34]:
import pandas as pd

# Display all columns
pd.set_option("display.max_columns", None)

# Display all rows
pd.set_option("display.max_rows", None)

# Prevent truncation of column values
pd.set_option("display.max_colwidth", None)

# Display full DataFrame (especially useful in Jupyter Notebooks)
pd.set_option("display.expand_frame_repr", False)

In [2]:
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11/"  # Update this path if needed

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/24 11:46:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pyspark
from pyspark.sql import Column, DataFrame, SparkSession, Window

spark = SparkSession.builder.getOrCreate()

In [6]:
import pandas as pd
from scipy.stats import zscore

In [22]:
import pandas as pd
import numpy as np

def validate_dataframe(df: pd.DataFrame, outlier_cols=None, category_checks=None, range_check_cols=None):
    """
    Perform advanced QC checks on a DataFrame and return results in a structured DataFrame.

    Parameters:
    - df (pd.DataFrame): The input dataframe to validate.
    - outlier_cols (list, optional): Columns to check for outliers (numeric only).
    - category_checks (dict, optional): Dictionary where keys are categorical columns, and values are lists of expected categories.
    - range_check_cols (dict, optional): Dictionary where keys are columns, and values are (min, max) tuples for valid ranges.

    Returns:
    - pd.DataFrame: QC results for each column.
    """
    
    def check_outliers(series):
        """Detects outliers using the IQR method, only for numeric columns."""
        if not pd.api.types.is_numeric_dtype(series):
            return None  # Skip non-numeric columns
        q1, q3 = np.percentile(series.dropna(), [25, 75])
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        return ((series < lower_bound) | (series > upper_bound)).sum()

    def check_unexpected_categories(series, expected_values):
        """Check for unexpected categories in a categorical column."""
        if series.dtype.name == "object" or pd.api.types.is_categorical_dtype(series):
            return list(set(series.dropna().unique()) - set(expected_values))
        return None

    def check_value_ranges(series, min_val, max_val):
        """Check for out-of-range values, only for numeric columns."""
        if not pd.api.types.is_numeric_dtype(series):
            return None  # Skip non-numeric columns
        return ((series < min_val) | (series > max_val)).sum()

    qc_results = {
        "Column": df.columns.tolist(),
        "Missing_Values": df.isnull().sum().tolist(),
        "Fully_Null_Columns": [df[col].isnull().all() for col in df.columns],
        "Dtype": [df[col].dtype.name for col in df.columns],
        "Negative_Values": [(df[col] < 0).sum() if pd.api.types.is_numeric_dtype(df[col]) else None for col in df.columns],
        "Duplicate_Rows": [df.duplicated().sum() if i == 0 else None for i in range(len(df.columns))],
        "Outliers": [check_outliers(df[col]) if outlier_cols and col in outlier_cols else None for col in df.columns],
        "Unexpected_Categories": [check_unexpected_categories(df[col], category_checks[col]) if category_checks and col in category_checks else None for col in df.columns],
        "Out_of_Range_Values": [check_value_ranges(df[col], *range_check_cols[col]) if range_check_cols and col in range_check_cols else None for col in df.columns],
    }
    
    return pd.DataFrame(qc_results)
    



## Patient Gender:

In [18]:
import pandas as pd
patient_gender = pd.read_csv('/Users/richa_srivastava/new_KOHLS/price-optimizer/personal/Century Health Assignment/test_data/patient_gender.csv')
display(patient_gender.head(5))

,Id,GENDER
0,54f1059e-6250-3949-6dd0-1dda9b85d22a,F
1,92675303-ca5b-136a-169b-e764c5753f06,M
2,a0b63e97-b6fd-5fe1-8f2d-2bec915efa97,M
3,abc59f62-dc5a-5095-1141-80b4ee8be73b,F
4,28d7b56c-6056-d0a2-2991-39d6e917216c,F


In [19]:
patient_gender.columns

Index(['Id', 'GENDER'], dtype='object')

In [23]:
display(validate_dataframe(patient_gender, outlier_cols=['GENDER'], category_checks=None, range_check_cols=None))

,Column,Missing_Values,Fully_Null_Columns,Dtype,Negative_Values,Duplicate_Rows,Outliers,Unexpected_Categories,Out_of_Range_Values
0,Id,0,False,object,None,0.0,None,None,None
1,GENDER,0,False,object,None,NaN,None,None,None


## Medications

In [24]:
import pandas as pd
medications = pd.read_csv('/Users/richa_srivastava/new_KOHLS/price-optimizer/personal/Century Health Assignment/test_data/medications.csv')
display(medications.head(5))

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2020-04-22T02:47:09Z,NaN,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,402DFAEB-1888-F562-4651-F75203EB1E28,849574,Naproxen sodium 220 MG Oral Tablet,274.28,0.0,45,12342.60,200936003.0,Lupus erythematosus
1,2020-04-22T02:47:09Z,2020-05-17T02:47:09Z,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,402DFAEB-1888-F562-4651-F75203EB1E28,312615,predniSONE 20 MG Oral Tablet,16.04,0.0,1,16.04,200936003.0,Lupus erythematosus
2,2021-08-01T02:47:09Z,2021-08-15T02:47:09Z,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,FBC2F3B3-AB93-707E-AD9E-337990345F32,241834,cycloSPORINE modified 100 MG Oral Capsule,473.61,0.0,1,473.61,200936003.0,Lupus erythematosus
3,2021-08-01T02:47:09Z,2021-08-15T02:47:09Z,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,FBC2F3B3-AB93-707E-AD9E-337990345F32,312615,predniSONE 20 MG Oral Tablet,8.21,0.0,1,8.21,200936003.0,Lupus erythematosus
4,2023-05-18T02:47:09Z,2023-06-14T02:47:09Z,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,E3989F64-4D5B-32E0-9EDC-7874A02F45E8,241834,cycloSPORINE modified 100 MG Oral Capsule,408.04,0.0,1,408.04,200936003.0,Lupus erythematosus


In [ ]:
medications.columns

Index(['START', 'STOP', 'PATIENT', 'PAYER', 'ENCOUNTER', 'CODE', 'DESCRIPTION',
       'BASE_COST', 'PAYER_COVERAGE', 'DISPENSES', 'TOTALCOST', 'REASONCODE',
       'REASONDESCRIPTION'],
      dtype='object')

In [28]:
medications['DESCRIPTION'].unique()

array(['Naproxen sodium 220 MG Oral Tablet',
       'predniSONE 20 MG Oral Tablet',
       'cycloSPORINE  modified 100 MG Oral Capsule',
       'CYCLOSPORINE  MODIFIED 100 MG ORAL CAPSULE',
       'NAPROXEN SODIUM 220 MG ORAL TABLET',
       'PREDNISONE 20 MG ORAL TABLET', 'prednisone 20 mg oral tablet',
       'Vitamin B12 5 MG/ML Injectable Solution',
       'ferrous sulfate 325 MG Oral Tablet',
       'naproxen sodium 220 mg oral tablet',
       'cyclosporine  modified 100 mg oral capsule',
       'ferrous sulfate 325 mg oral tablet',
       'FERROUS SULFATE 325 MG ORAL TABLET',
       'VITAMIN B12 5 MG/ML INJECTABLE SOLUTION',
       'vitamin b12 5 mg/ml injectable solution'], dtype=object)

In [33]:
display(validate_dataframe(medications, outlier_cols=['BASE_COST', 'DISPENSES'], category_checks={
    'REASONDESCRIPTION': ['Lupus erythematosus'], 
    'DESCRIPTION' :[
        'cyclosporine  modified 100 mg oral capsule',
        'Vitamin B12 5 MG/ML Injectable Solution',
        'naproxen sodium 220 mg oral tablet',
        'PREDNISONE 20 MG ORAL TABLET',
        'predniSONE 20 MG Oral Tablet',
        'cycloSPORINE  modified 100 MG Oral Capsule',
        'CYCLOSPORINE  MODIFIED 100 MG ORAL CAPSULE',
        'NAPROXEN SODIUM 220 MG ORAL TABLET']
    }, range_check_cols=None))

,Column,Missing_Values,Fully_Null_Columns,Dtype,Negative_Values,Duplicate_Rows,Outliers,Unexpected_Categories,Out_of_Range_Values
0,START,0,False,object,NaN,0.0,NaN,None,None
1,STOP,3219,False,object,NaN,NaN,NaN,None,None
2,PATIENT,0,False,object,NaN,NaN,NaN,None,None
3,PAYER,0,False,object,NaN,NaN,NaN,None,None
4,ENCOUNTER,0,False,object,NaN,NaN,NaN,None,None
5,CODE,0,False,int64,0.0,NaN,NaN,None,None
6,DESCRIPTION,0,False,object,NaN,NaN,NaN,"[FERROUS SULFATE 325 MG ORAL TABLET, Naproxen sodium 220 MG Oral Tablet, ferrous sulfate 325 MG Oral Tablet, VITAMIN B12 5 MG/ML INJECTABLE SOLUTION, prednisone 20 mg oral tablet, ferrous sulfate 325 mg oral tablet, vitamin b12 5 mg/ml injectable solution]",None
7,BASE_COST,0,False,float64,0.0,NaN,0.0,None,None
8,PAYER_COVERAGE,0,False,float64,0.0,NaN,NaN,None,None
9,DISPENSES,0,False,int64,0.0,NaN,3077.0,None,None


## Conditions

In [ ]:
import pandas as pd
conditions = pd.read_excel("/Users/richa_srivastava/new_KOHLS/price-optimizer/personal/Century Health Assignment/test_data/conditions.xlsx", sheet_name=0)  # 0 refers to the first sheet

display(conditions.head(5))

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2020-04-21,NaN,28D7B56C-6056-D0A2-2991-39D6E917216C,402dfaeb-1888-f562-4651-f75203eb1e28,200936003,Lupus erythematosus
1,2001-11-07,NaN,239AE86A-96DB-6211-9042-D3F2850AABB8,10d39d6f-7d98-8d03-8866-0a7cd363890b,200936003,Lupus erythematosus
2,2001-11-07,NaN,239AE86A-96DB-6211-9042-D3F2850AABB8,10d39d6f-7d98-8d03-8866-0a7cd363890b,271737000,Anemia (disorder)
3,2017-11-05,NaN,6C434506-FB4B-3E3F-C19D-553DEC3B6C17,add4fa37-cabe-a5aa-f9da-8117464bad04,200936003,Lupus erythematosus
4,1989-09-15,NaN,6872DEF5-772F-427C-3053-DE6E1C71CE0A,d8a684cb-6ce1-5909-59e7-3422469d18d0,200936003,Lupus erythematosus


In [ ]:
conditions.columns

Index(['START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION'], dtype='object')

## Patients

In [12]:
patients = pd.read_csv("/Users/richa_srivastava/new_KOHLS/price-optimizer/personal/Century Health Assignment/test_data/patients.csv")  # 0 refers to the first sheet

display(patients.head(5))

,PATIENT_ID,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
0,54f1059e-6250-3949-6dd0-1dda9b85d22a,2002-01-21,NaN,999-28-3364,S99987398,X44428214X,Ms.,Fredricka415,Crist667,NaN,...,Fort Worth,Texas,Tarrant County,48439.0,76104,0.329154,-97.399553,9990.08,0.0,35460
1,92675303-ca5b-136a-169b-e764c5753f06,1997-04-16,NaN,999-62-9859,S99999770,X75016560X,Mr.,Lorenzo669,Urrutia540,NaN,...,Arlington,Texas,Tarrant County,48439.0,76006,0.328145,-97.068885,10936.80,0.0,88407
2,a0b63e97-b6fd-5fe1-8f2d-2bec915efa97,1993-02-04,NaN,999-53-6488,S99979170,X60079936X,Mr.,Luther918,MacGyver246,NaN,...,Houston,Texas,Harris County,48339.0,77357,0.300834,-95.649706,10662.16,0.0,41800
3,abc59f62-dc5a-5095-1141-80b4ee8be73b,1995-05-23,NaN,999-37-1058,S99981031,X14759314X,Mrs.,Jacque955,Jones311,NaN,...,Waco,Texas,McLennan County,48309.0,76655,0.315173,-97.292558,11455.93,0.0,41915
4,28d7b56c-6056-d0a2-2991-39d6e917216c,1993-11-13,NaN,999-31-6091,S99958903,X63033472X,Mrs.,Angela104,Stanton715,NaN,...,Levelland,Texas,Hockley County,48219.0,79336,0.337037,-102.361829,12449.11,0.0,67198


,Column,Missing_Values,Fully_Null_Columns,Dtype,Negative_Values,Duplicate_Rows,Outliers,Unexpected_Categories,Out of Range Values
0,PATIENT_ID,0,False,object,NaN,0.0,None,None,"{'DEATHDATE': None, 'GENDER': None, 'FIPS': No..."
1,BIRTHDATE,0,False,object,NaN,NaN,None,None,None
2,DEATHDATE,10000,True,float64,0.0,NaN,None,None,None
3,SSN,0,False,object,NaN,NaN,None,None,None
4,DRIVERS,2320,False,object,NaN,NaN,None,None,None
5,PASSPORT,2961,False,object,NaN,NaN,None,None,None
6,PREFIX,2663,False,object,NaN,NaN,None,None,None
7,FIRST,0,False,object,NaN,NaN,None,None,None
8,LAST,0,False,object,NaN,NaN,None,None,None
9,SUFFIX,9941,False,object,NaN,NaN,None,None,None


In [ ]:
patients.columns

Index(['PATIENT_ID', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT',
       'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE',
       'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE',
       'COUNTY', 'FIPS', 'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES',
       'HEALTHCARE_COVERAGE', 'INCOME'],
      dtype='object')

In [ ]:
encounters.dtypes

Id                      object
START                   object
STOP                    object
PATIENT                 object
ORGANIZATION            object
PROVIDER                object
PAYER                   object
ENCOUNTERCLASS          object
CODE                     int64
DESCRIPTION             object
BASE_ENCOUNTER_COST    float64
TOTAL_CLAIM_COST       float64
PAYER_COVERAGE         float64
REASONCODE             float64
REASONDESCRIPTION       object
dtype: object

## Encounters

In [ ]:
encounters = pd.read_parquet("/Users/richa_srivastava/new_KOHLS/price-optimizer/personal/Century Health Assignment/test_data/encounters.parquet")  # 0 refers to the first sheet

display(encounters.head(5))

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,6539c675-dcf2-cace-a90d-7f097d1799b3,2014-02-10T11:00:08Z,2014-02-10T11:15:08Z,54F1059E-6250-3949-6DD0-1DDA9B85D22A,ef42ec52-a1c1-3fc5-8e52-da46f7f3970c,35088bec-f756-35d8-8140-b2b017c0eacc,8fa6c185-e44e-3e34-8bd8-39be8694f4ce,wellness,410620009,Well child visit (procedure),133.36,405.36,0.0,NaN,None
1,748a443c-9819-4f3a-8648-0096ace6d3fd,2014-06-04T05:19:43Z,2014-06-04T05:34:43Z,92675303-CA5B-136A-169B-E764C5753F06,02078b23-a943-3175-b5f0-8d386dd8eb56,af22b73b-d38f-3d92-8a95-bd57ac2dc3f1,b046940f-1664-3047-bca7-dfa76be352a4,wellness,410620009,Well child visit (procedure),133.36,269.36,0.0,NaN,None
2,678e03b1-21ac-fbfa-3506-17bee72a6335,2015-06-10T05:19:43Z,2015-06-10T05:34:43Z,92675303-CA5B-136A-169B-E764C5753F06,02078b23-a943-3175-b5f0-8d386dd8eb56,af22b73b-d38f-3d92-8a95-bd57ac2dc3f1,b046940f-1664-3047-bca7-dfa76be352a4,wellness,162673000,General examination of patient (procedure),133.36,269.36,0.0,NaN,None
3,45987160-8815-858e-893b-6490a63e919c,2015-04-10T03:27:04Z,2015-04-10T03:42:04Z,A0B63E97-B6FD-5FE1-8F2D-2BEC915EFA97,44f0ad91-f33b-3fe5-ba4f-868503d593fe,a79c71d6-e0b4-3450-a0e1-ee08146aeb74,734afbd6-4794-363b-9bc0-6a3981533ed5,wellness,162673000,General examination of patient (procedure),133.36,677.36,0.0,NaN,None
4,c56e3b19-5a51-0fc8-a82e-8bf89575ca71,2016-06-15T05:19:43Z,2016-06-15T05:34:43Z,92675303-CA5B-136A-169B-E764C5753F06,02078b23-a943-3175-b5f0-8d386dd8eb56,af22b73b-d38f-3d92-8a95-bd57ac2dc3f1,b046940f-1664-3047-bca7-dfa76be352a4,wellness,162673000,General examination of patient (procedure),133.36,269.36,0.0,NaN,None


In [ ]:
encounters.columns

Index(['Id', 'START', 'STOP', 'PATIENT', 'ORGANIZATION', 'PROVIDER', 'PAYER',
       'ENCOUNTERCLASS', 'CODE', 'DESCRIPTION', 'BASE_ENCOUNTER_COST',
       'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'REASONCODE',
       'REASONDESCRIPTION'],
      dtype='object')

## Symptoms

In [14]:
symptoms = pd.read_csv("/Users/richa_srivastava/new_KOHLS/price-optimizer/personal/Century Health Assignment/test_data/symptoms.csv")  # 0 refers to the first sheet

display(symptoms.head(5))

,PATIENT,GENDER,RACE,ETHNICITY,AGE_BEGIN,AGE_END,PATHOLOGY,NUM_SYMPTOMS,SYMPTOMS
0,28d7b56c-6056-d0a2-2991-39d6e917216c,NaN,white,hispanic,26,NaN,Lupus erythematosus,4,Rash:34;Joint Pain:39;Fatigue:9;Fever:12
1,6c434506-fb4b-3e3f-c19d-553dec3b6c17,NaN,black,hispanic,58,NaN,Lupus erythematosus,4,Rash:19;Joint Pain:44;Fatigue:48;Fever:15
2,44a8ca45-6c6e-38bb-fac0-ddbf7a7ee3a4,NaN,white,nonhispanic,50,NaN,Lupus erythematosus,4,Rash:2;Joint Pain:32;Fatigue:12;Fever:6
3,780ec78c-22a0-fcdb-17c6-ae9b2fcace9c,NaN,white,hispanic,39,NaN,Lupus erythematosus,4,Rash:30;Joint Pain:30;Fatigue:41;Fever:19
4,cf5956bb-34f2-841b-2505-57b99991c377,NaN,white,nonhispanic,29,NaN,Lupus erythematosus,3,Rash:28;Joint Pain:26;Fatigue:33;Fever:5


In [15]:
display(validate_dataframe(symptoms))

AttributeError: module 'numpy' has no attribute 'iterable'

In [ ]:
symptoms.columns

Index(['PATIENT', 'GENDER', 'RACE', 'ETHNICITY', 'AGE_BEGIN', 'AGE_END',
       'PATHOLOGY', 'NUM_SYMPTOMS', 'SYMPTOMS'],
      dtype='object')

In [ ]:
symptoms.count()

PATIENT         1423
GENDER             0
RACE            1423
ETHNICITY       1423
AGE_BEGIN       1423
AGE_END            0
PATHOLOGY       1423
NUM_SYMPTOMS    1423
SYMPTOMS        1423
dtype: int64

In [ ]:
df.count()

PATIENT         1423
GENDER             0
RACE            1423
ETHNICITY       1423
AGE_BEGIN       1423
AGE_END            0
PATHOLOGY       1423
NUM_SYMPTOMS    1423
Rash            1423
Joint Pain      1423
Fatigue         1423
Fever           1423
dtype: int64

In [ ]:
symptom_dicts = symptoms["SYMPTOMS"].apply(lambda x: dict(item.split(":") for item in x.split(";")))

# Create a new DataFrame from the extracted symptoms
symptom_df = pd.DataFrame(symptom_dicts.tolist()).astype(float)

# Merge with original DataFrame
df = pd.concat([symptoms.drop(columns=["SYMPTOMS"]), symptom_df], axis=1)
df = df.sort_values(by="PATIENT", ascending=True)
display(df.head(5))

,PATIENT,GENDER,RACE,ETHNICITY,AGE_BEGIN,AGE_END,PATHOLOGY,NUM_SYMPTOMS,Rash,Joint Pain,Fatigue,Fever
21,0005a479-d128-178d-4aac-fefe1b0cc77e,NaN,white,nonhispanic,34,NaN,Lupus erythematosus,4,12.0,37.0,5.0,7.0
86,0033e7f5-91d2-4997-6dab-aee714b45330,NaN,white,nonhispanic,33,NaN,Lupus erythematosus,4,7.0,31.0,19.0,2.0
52,00cbb03e-9551-3437-fd0d-8a98d4fb87c4,NaN,white,nonhispanic,31,NaN,Anemia (disorder),4,21.0,36.0,29.0,3.0
51,00cbb03e-9551-3437-fd0d-8a98d4fb87c4,NaN,white,nonhispanic,31,NaN,Lupus erythematosus,3,21.0,36.0,29.0,3.0
1257,012827de-3a30-41fb-95e1-b7840ff35d3c,NaN,asian,nonhispanic,45,NaN,Lupus erythematosus,4,1.0,3.0,35.0,15.0


25/02/23 23:05:25 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 841726 ms exceeds timeout 120000 ms
25/02/23 23:05:25 WARN SparkContext: Killing executors is not supported by current scheduler.
25/02/23 23:05:26 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
symptoms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423 entries, 0 to 1422
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PATIENT       1423 non-null   object 
 1   GENDER        0 non-null      float64
 2   RACE          1423 non-null   object 
 3   ETHNICITY     1423 non-null   object 
 4   AGE_BEGIN     1423 non-null   int64  
 5   AGE_END       0 non-null      float64
 6   PATHOLOGY     1423 non-null   object 
 7   NUM_SYMPTOMS  1423 non-null   int64  
 8   SYMPTOMS      1423 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 100.2+ KB


In [ ]:
patients = patients.rename(columns={"PATIENT_ID": "PATIENT"})  # Remove inplace=True

merged_df = (
    patients
    .merge(encounters, on="PATIENT", how="left")
    .merge(symptoms, on="PATIENT", how="left")
    .merge(conditions, on="PATIENT", how="left")
    .merge(medications, on="PATIENT", how="left")
)

display(merged_df)  # Use print(merged_df) if not in Jupyter/Databricks


,PATIENT,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,PAYER_y,ENCOUNTER_y,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE_y,DISPENSES,TOTALCOST,REASONCODE_y,REASONDESCRIPTION_y
0,54f1059e-6250-3949-6dd0-1dda9b85d22a,2002-01-21,NaN,999-28-3364,S99987398,X44428214X,Ms.,Fredricka415,Crist667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,92675303-ca5b-136a-169b-e764c5753f06,1997-04-16,NaN,999-62-9859,S99999770,X75016560X,Mr.,Lorenzo669,Urrutia540,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a0b63e97-b6fd-5fe1-8f2d-2bec915efa97,1993-02-04,NaN,999-53-6488,S99979170,X60079936X,Mr.,Luther918,MacGyver246,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abc59f62-dc5a-5095-1141-80b4ee8be73b,1995-05-23,NaN,999-37-1058,S99981031,X14759314X,Mrs.,Jacque955,Jones311,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28d7b56c-6056-d0a2-2991-39d6e917216c,1993-11-13,NaN,999-31-6091,S99958903,X63033472X,Mrs.,Angela104,Stanton715,NaN,...,0133f751-9229-3cfd-815f-b6d4979bdd6a,402DFAEB-1888-F562-4651-F75203EB1E28,849574.0,Naproxen sodium 220 MG Oral Tablet,274.28,0.0,45.0,12342.60,200936003.0,Lupus erythematosus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32020,8d116ace-7fb6-7d5f-b86b-94547cbd796d,1968-03-07,NaN,999-22-1771,S99910062,X65201727X,Mr.,Andrew29,Pfannerstill264,NaN,...,8fa6c185-e44e-3e34-8bd8-39be8694f4ce,49E9369B-EF5E-22CB-5F7C-B5E3035841D2,312615.0,predniSONE 20 MG Oral Tablet,12.24,0.0,1.0,12.24,200936003.0,Lupus erythematosus
32021,8d116ace-7fb6-7d5f-b86b-94547cbd796d,1968-03-07,NaN,999-22-1771,S99910062,X65201727X,Mr.,Andrew29,Pfannerstill264,NaN,...,734afbd6-4794-363b-9bc0-6a3981533ed5,DA11A1E4-491D-50AC-2704-C80862EF10D9,241834.0,cyclosporine modified 100 mg oral capsule,265.17,0.0,1.0,265.17,200936003.0,Lupus erythematosus
32022,8d116ace-7fb6-7d5f-b86b-94547cbd796d,1968-03-07,NaN,999-22-1771,S99910062,X65201727X,Mr.,Andrew29,Pfannerstill264,NaN,...,734afbd6-4794-363b-9bc0-6a3981533ed5,DA11A1E4-491D-50AC-2704-C80862EF10D9,312615.0,predniSONE 20 MG Oral Tablet,13.37,0.0,1.0,13.37,200936003.0,Lupus erythematosus
32023,8d116ace-7fb6-7d5f-b86b-94547cbd796d,1968-03-07,NaN,999-22-1771,S99910062,X65201727X,Mr.,Andrew29,Pfannerstill264,NaN,...,734afbd6-4794-363b-9bc0-6a3981533ed5,48BD83C4-5F36-4BD4-A8AC-1E0A5D1AFC2D,241834.0,cycloSPORINE modified 100 MG Oral Capsule,233.49,0.0,1.0,233.49,200936003.0,Lupus erythematosus


In [ ]:
import pandas as pd

# Ensure Pandas displays all rows (even for dtypes)
pd.set_option("display.max_rows", None)

print(merged_df.dtypes)

PATIENT                 object
BIRTHDATE               object
DEATHDATE              float64
SSN                     object
DRIVERS                 object
PASSPORT                object
PREFIX                  object
FIRST                   object
LAST                    object
SUFFIX                  object
MAIDEN                  object
MARITAL                 object
RACE_x                  object
ETHNICITY_x             object
GENDER_x               float64
BIRTHPLACE              object
ADDRESS                 object
CITY                    object
STATE                   object
COUNTY                  object
FIPS                   float64
ZIP                      int64
LAT                    float64
LON                    float64
HEALTHCARE_EXPENSES    float64
HEALTHCARE_COVERAGE    float64
INCOME                   int64
Id                      object
START_x                 object
STOP_x                  object
ORGANIZATION            object
PROVIDER                object
PAYER_x 

In [ ]:



symptoms_QC = pd.read_csv("/Users/richa_srivastava/new_KOHLS/price-optimizer/century-health-assignment/data/01_raw/symptoms_QC.csv")  # 0 refers to the first sheet
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None) 
display(symptoms_QC.drop_duplicates().head(150))

,Column,Missing_Values,Fully_Null_Columns,Dtype,Negative_Values,Duplicate_Rows,Outliers,Unexpected_Categories,Age_Range_Issues
0,PATIENT,0,False,object,NaN,0.0,NaN,NaN,"{'GENDER': None, 'AGE_BEGIN': [], 'AGE_END': None, 'NUM_SYMPTOMS': []}"
1,GENDER,1423,True,float64,0.0,NaN,NaN,NaN,NaN
2,RACE,0,False,object,NaN,NaN,NaN,"['white', 'black', 'native', 'other', 'hawaiian', 'asian']",NaN
3,ETHNICITY,0,False,object,NaN,NaN,NaN,NaN,NaN
4,AGE_BEGIN,0,False,int64,0.0,NaN,[],NaN,NaN
5,AGE_END,1423,True,float64,0.0,NaN,NaN,NaN,NaN
6,PATHOLOGY,0,False,object,NaN,NaN,NaN,NaN,NaN
7,NUM_SYMPTOMS,0,False,int64,0.0,NaN,[],NaN,NaN
8,SYMPTOMS,0,False,object,NaN,NaN,NaN,NaN,NaN
